# Infegy's API Discussion
Walking you through just how easy it is to get started with Infegy Starscape's API

## Import Statements
We're using well-known, open-source libraries that are all free!

In [278]:
import requests
import json

import plotly.express as px

import folium
from folium import plugins
import folium.plugins as plugins
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

## Authentication
We access our API via a standard bearer token. 

In [279]:
def read_token():
    f = open("bearerToken.txt", "r")

    bearer_token = f.readline()
    f.close()
    return bearer_token

bearer_token = read_token()

## Aggregate Endpoint Agg
This is one of the three endpoints that we offer. Aggregations collects and counts results for a specific query. This can do standard things: counting how many posts mention Apple by day or asking how positive or negative the conversation is around Apple for a certain time period. 

It also can do wild, customized aggregations. Some examples - which country has the highest level of Trust towards Apple products or has the age of the conversation shifted over time? 



## Standardized Query Structure
Here's what a query looks like. These have two parts: First the actual query - the type of content you're looking for. 

Secondly, they have an aggregator section. This allows you tell tell Infegy specifically how you want to count the attribute you're looking for. 

In [280]:
query = {
  "op": "and",
  "values": [
    {
      "op": "or",
      "values": [
        {
          "op": "contains",
          "field": "body",
          "value": "Apple"
        },
        {
          "op": "contains",
          "field": "entities",
          "value": "Od"
        },
      ]
    },
    {
      "op": "=",
      "field": "language",
      "value": "en"
    },
    {
      "op": ">=",
      "field": "published",
      "value": "2020-01-01"
    }
  ]
}

aggs = {
    "weekly histogram": {
      "op": "histogram",
      "field": "published",
      "interval": "week"
    },
    "sentiment totals": {
      "op": "keyword",
      "field": "sentiment"
    }
  }

In [281]:
def send_query(query, aggs, aggregator, bearer_token, endpoint, flatten):
    
    url = "https://starscape.infegy.com/api/query/" + endpoint + "?flatten=" + flatten + "&include_total=true&format=json&fill_with_zero=true"
    
    payload = {
        "dataset_id": "ds_gj4u3F40SLa",
        "timezone": "Etc/UTC",
        "query": query,
        aggregator: aggs
    }
    headers = {
        "Content-Type": "application/json",
        "Authorization": bearer_token
    }

    response = requests.post(url, json=payload, headers=headers)
    
    response.raise_for_status()
    
    return response.json()

response = send_query(query, aggs, "aggs", bearer_token, "agg", "true")

In [282]:
def parse_volume_response(response):
    labels = response['weekly histogram']['labels']
    counts = response['weekly histogram']['series'][0]['values']
    
    return labels, counts
    
labels, counts = parse_volume_response(response)

def parse_sentiment_response(response):
    positive = response['sentiment totals']['series'][0]['values'][0]
    negative = response['sentiment totals']['series'][0]['values'][1]
    neutral = response['sentiment totals']['series'][0]['values'][2]

    return [positive, negative, neutral]

sentiment_list = parse_sentiment_response(response)

In [283]:
fig = px.line(x=labels, y=counts, title='Apple Post Volume')
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Record Count",
    font=dict(
        family="Roboto",
        size=15,
        color="#005184"
    )
)

fig['data'][0]['line']['color']='#22aaff'

fig.show()

fig = px.bar(x=['positive', 'negative', 'neutral'], y=sentiment_list, title="Apple's Sentiment", labels={'x': 'Year', 'y':'Net Sentiment'})
fig.update_layout(
    xaxis_title="Category",
    yaxis_title="Record Count",
    font=dict(
        family="Roboto",
        size=15,
        color="#005184"
    )
)

fig.show()


## Building a Changing Sentiment Chart
Now, let's build a chart that shows how Apple's sentiment changes over time. We can reuse our existing query from before, but we'll change our aggregators to reflect what we want. 

In [284]:
aggs = {
    "weekly histogram": {
      "op": "histogram",
      "field": "published",
      "interval": "week",
        "aggs" :{
            "sentiment": {
                "op": "keyword",
                "field": "sentiment",
                "filter": ['p', 'n', '0'],
                "aggs": {
                    "%": {
                        "op": "percent"
                    }
                }
            }
        }
    },
  }

response = send_query(query, aggs, "aggs", bearer_token, "agg", "true")

In [285]:
def parse_histogram_sentiment_response(response):
    labels = response['weekly histogram']['labels']
    positive = response['weekly histogram']['series'][4]['values']
    negative = response['weekly histogram']['series'][5]['values']
    neutral = response['weekly histogram']['series'][6]['values']
    
    return labels, positive, negative, neutral
    
labels, positive, negative, neutral = parse_histogram_sentiment_response(response)

In [286]:
fig = px.line(x=labels, y=[positive, negative, neutral], title="Apple's Net Sentiment")

series_names = {'wide_variable_0':'Positive', 'wide_variable_1': 'Negative', 'wide_variable_2': 'Neutral'}
fig.for_each_trace(lambda t: t.update(name = series_names[t.name],
                                      legendgroup = series_names[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, series_names[t.name]), 
                                     )
                  )

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Net Sentiment",
    legend_title="Legend",
    font=dict(
        family="Roboto",
        size=15,
        color="#005184"
    )
)

fig['data'][0]['line']['color']='#88dd11'
fig['data'][1]['line']['color']='#ff3366'
fig['data'][2]['line']['color']='#8899bb'


fig.show()


## Let's do dive deeper into aggregations
So far, we've been showing you things you can see within any normal type of social listening tool. Now, we'll dive into customized aggregations to take a look at customized metrics you can't get anywhere else.

Let's look at how social users talking about Apple change over time. 

In [287]:
aggs = {
    "emotions": {
        "op": "keyword",
        "field": "emotions",
        "aggs": {
            "source_gender": {
                "op": "keyword",
                "field": "gender",
                "aggs": {
                    "%": {
                        "op": 'percent'
                    }
                }
                
            }
        }
    },
}

response = send_query(query, aggs, "aggs", bearer_token, "agg", "true")

def parse_age_response_numbers(response):
    
    labels = response['emotions']['labels']
    labels = [label.title() for label in labels]
    male = response['emotions']['series'][1]['values']
    female = response['emotions']['series'][2]['values']
    non_binary = response['emotions']['series'][3]['values']
    
    return labels, male, female, non_binary

labels, male, female, non_binary = parse_age_response(response)

fig = px.bar(y = labels, x=[male, female, non_binary], title="Emotions by Gender")

series_names = {'wide_variable_0':'Male', 'wide_variable_1': 'Female', 'wide_variable_2': 'Non-Binary'}
fig.for_each_trace(lambda t: t.update(name = series_names[t.name],
                                      legendgroup = series_names[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, series_names[t.name]), 
                                     )
                  )

fig.update_layout(
    xaxis_title="Number of Posts",
    yaxis_title="Emotions",
    legend_title="Legend",
    font=dict(
        family="Roboto",
        size=15,
        color="#005184"
    )
)

fig.show()




## Building a Heatmap with Folium and Infegy Starscape's API
Now, we'll take a look at a totally different endpoint - the records endpoint. With this, we can download individual posts with latitude and longitude points. We can display that with Folium, a Python-based Leaflet wrapper. 

In [288]:
records = { 
        "size": 100000, 
        "fields": ["geohash"]
}

query = {
  "op": "and",
  "values": [
    {
      "op": "or",
      "values": [
        {
          "op": "contains",
          "field": "body",
          "value": "Apple"
        },
        {
          "op": "contains",
          "field": "entities",
          "value": "Od"
        },
      ]
    },
    {
      "op": "=",
      "field": "language",
      "value": "en"
    },
    {
      "op": ">=",
      "field": "published",
      "value": "2020-01-01"
    },
    {
      "op": "not",
      "values": [
        {
          "op": "contains",
          "value": None,
          "field": 'geohash'
        }
      ]
    }
  ]
}

response = send_query(query, records, "records", bearer_token, "records", "true")

folium_map = folium.Map(location=[0, 0], zoom_start = 2)

HeatMap(data=response['records'], radius=10, max_zoom=10).add_to(folium_map)

folium_map
